<a href="https://colab.research.google.com/github/ibenoam/Neural_cryptography/blob/main/Neural_Cryptography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural-Cryptography:
### Solving the key exchange problem by synchronize Tree Parity Machines

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
N = 1000 # each N input neurons goes to 1 hidden neuron
L = 3 # the "synaptic" depth
K = 3 # number of perceptrons (or hidden layer size)
M = 1 # the input consists of integers between -M to M (without zero)

num_sys = 100 # we average each result "num_sys" times

step_converage = np.zeros((1,num_sys))
sync_time = np.zeros((1,num_sys))
attacker_time = np.zeros((1,num_sys))

In [ ]:
class TPM:
  def __init__(self,N,K,L,M=1):
    # Constants:
    self.N = N
    self.K = K
    self.L = L
    self.M = M

    # Initialization:
    self.w = np.floor((2*L+1) * np.random.rand(N,K)) - L
    self.init_w = self.w
    self.fields = np.zeros((1,K))
    self.sigmas = np.zeros((1,K))
    self.tau = 0

    # The feed-forward process:
  def forward(self,input):
    self.fields = np.sum(self.w*input, axis=0)
    self.sigmas = np.sign(self.fields)
    self.tau = np.prod(self.sigmas)
  
    # updating the weights:
  def update(self,input):
    update_indexes = np.where(self.sigmas == self.tau)
    self.w[:,update_indexes] += input[:,update_indexes]
    self.w[np.abs(self.w)>self.L] = np.sign(self.w[np.abs(self.w)>self.L])*self.L


In [ ]:
for i in range(num_sys):

    # Creates the objects from the "TPM" class:
    alice = TPM(N,K,L)
    bob = TPM(N,K,L)
    # Creates the attacker's TPM:
    eve = TPM(N,K,L)

    when_move = np.array([])
    corr_alice_bob = np.array([])
    count_steps = 0
    while not np.array_equal(alice.w,bob.w):
        # print("After {} steps.".format(count_steps))
        count_steps += 1
        inputs = np.sign(np.random.rand(N,K)-0.5)
        alice.forward(inputs)
        bob.forward(inputs)
    
        # According to the algorithm, Alice and Bob take a step only when their outputs are identical:
        if alice.tau == bob.tau:
            when_move=np.append(when_move,1)
            alice.update(inputs)
            bob.update(inputs)
            # corr_alice_bob=np.append(corr_alice_bob,np.corrcoef(alice.w[:], bob.w[:]))
        else:
            when_move=np.append(when_move,0)
        
        ''' The best strategy for an attacker with only 1 TPM:
        when Eve gets the same output like Alice- she updates the weights according to the same learning rule.
        But, because of the fact that she cannot influence the dynamics between Alice and Bob, she has to do something
        even when her output is different, otherwise she does not have a real chance.
        So, she looks for the perceptron with the minimal absolute field and flips it (it has the less "confidence").
        Now, her output becomes identical to Alice's output and she can update according to the regular learning rule.
        '''
        if eve.tau == alice.tau:
            eve.update(inputs)
        else:
            min_field_index = np.argmin(np.abs(eve.fields))
            eve.sigmas[min_field_index] *= -1 # flips a field
            eve.tau *= -1 # now, flips the output
            eve.update(inputs)

    # print(" finished {}% of the runtime".format(100*(i+1)/num_sys))
    sync_time[0, i] = count_steps
print(" The average synchronization time between Alice and bob is {} steps with std of {}".format(np.mean(sync_time), np.std(sync_time)))

 The average synchronization time between Alice and bob is 418.75 steps with std of 132.59527706521072


In [ ]:
a = np.array([1, 2, 3, 4])
b = np.array([1, 2, 3, 4])
print(np.correlate(a, b))


In [ ]:
def entropy_calc(arr):
  

In [ ]:
print(sync_time)

[[0.]]
